# L5: Automate Event Planning

In this lesson, you will learn more about Tasks.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [2]:
from crewai import Agent, Crew, Task

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [3]:
import os
from utils import get_openai_api_key,get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [4]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

# Initialize the tools
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [5]:
# Agent 1: Venue Coordinator
venue_coordinator = Agent(
    role="Venue Coordinator",
    goal="Identify and book an appropriate venue "
    "based on event requirements",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "With a keen sense of space and "
        "understanding of event logistics, "
        "you excel at finding and securing "
        "the perfect venue that fits the event's theme, "
        "size, and budget constraints."
    )
)

In [6]:
 # Agent 2: Logistics Manager
logistics_manager = Agent(
    role='Logistics Manager',
    goal=(
        "Manage all logistics for the event "
        "including catering and equipmen"
    ),
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Organized and detail-oriented, "
        "you ensure that every logistical aspect of the event "
        "from catering to equipment setup "
        "is flawlessly executed to create a seamless experience."
    )
)

In [7]:
# Agent 3: Marketing and Communications Agent
marketing_communications_agent = Agent(
    role="Marketing and Communications Agent",
    goal="Effectively market the event and "
         "communicate with participants",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Creative and communicative, "
        "you craft compelling messages and "
        "engage with potential attendees "
        "to maximize event exposure and participation."
    )
)

## Creating Venue Pydantic Object

- Create a class `VenueDetails` using [Pydantic BaseModel](https://docs.pydantic.dev/latest/api/base_model/).
- Agents will populate this object with information about different venues by creating different instances of it.

In [8]:
from pydantic import BaseModel
# Define a Pydantic model for venue details 
# (demonstrating Output as Pydantic)
class VenueDetails(BaseModel):
    name: str
    address: str
    capacity: int
    booking_status: str

## Creating Tasks
- By using `output_json`, you can specify the structure of the output you want.
- By using `output_file`, you can get your output in a file.
- By setting `human_input=True`, the task will ask for human feedback (whether you like the results or not) before finalising it.

In [9]:
venue_task = Task(
    description="Find a venue in {event_city} "
                "that meets criteria for {event_topic}.",
    expected_output="All the details of a specifically chosen"
                    "venue you found to accommodate the event.",
    human_input=True,
    output_json=VenueDetails,
    output_file="venue_details.json",  
      # Outputs the venue details as a JSON file
    agent=venue_coordinator
)

- By setting `async_execution=True`, it means the task can run in parallel with the tasks which come after it.

In [10]:
logistics_task = Task(
    description="Coordinate catering and "
                 "equipment for an event "
                 "with {expected_participants} participants "
                 "on {tentative_date}.",
    expected_output="Confirmation of all logistics arrangements "
                    "including catering and equipment setup.",
    human_input=True,
    async_execution=True,
    agent=logistics_manager
)

In [11]:
marketing_task = Task(
    description="Promote the {event_topic} "
                "aiming to engage at least"
                "{expected_participants} potential attendees.",
    expected_output="Report on marketing activities "
                    "and attendee engagement formatted as markdown.",
    async_execution=True,
    output_file="marketing_report.md",  # Outputs the report as a text file
    agent=marketing_communications_agent
)

## Creating the Crew

**Note**: Since you set `async_execution=True` for `logistics_task` and `marketing_task` tasks, now the order for them does not matter in the `tasks` list.

In [12]:
# Define the crew with agents and tasks
event_management_crew = Crew(
    agents=[venue_coordinator, 
            logistics_manager, 
            marketing_communications_agent],
    
    tasks=[venue_task, 
           logistics_task, 
           marketing_task],
    
    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [13]:
event_details = {
    'event_topic': "Tech Innovation Conference",
    'event_description': "A gathering of tech innovators "
                         "and industry leaders "
                         "to explore future technologies.",
    'event_city': "San Francisco",
    'tentative_date': "2024-09-15",
    'expected_participants': 500,
    'budget': 20000,
    'venue_type': "Conference Hall"
}

**Note 1**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

**Note 2**: 
- Since you set `human_input=True` for some tasks, the execution will ask for your input before it finishes running.
- When it asks for feedback, use your mouse pointer to first click in the text box before typing anything.

In [14]:
result = event_management_crew.kickoff(inputs=event_details)

 [DEBUG]: == Working Agent: Venue Coordinator
 [INFO]: == Starting Task: Find a venue in San Francisco that meets criteria for Tech Innovation Conference.


> Entering new CrewAgentExecutor chain...
I need to start by researching potential venues in San Francisco that can accommodate a Tech Innovation Conference.

Action: Search the internet
Action Input: {"search_query": "venues for events in San Francisco"} 


Search results: Title: Top 20 Venues in San Francisco, CA - PartySlate
Link: https://www.partyslate.com/find-venues/near/san-francisco-ca-usa
Snippet: Julia Morgan Ballroom is the perfect venue to host an event of any sort in San Francisco. The large, raw space with eclectic decor is great for weddings ...
---
Title: Party Venues near San Francisco, CA - Eventective
Link: https://www.eventective.com/san-francisco-ca/party-event-venues/
Snippet: Dominican offers unique venues for meetings, workshops, seminars, conferences, as well as intimate memorials, weddings, concerts and pe

Final Answer: 
Based on the information from the Julia Morgan Ballroom website, the venue is located in downtown San Francisco and offers a total venue square footage of 15,500, with a ballroom square footage of 4,060. The room dimensions are 90'8"x44'9" with a ceiling height of 17'2". The venue can accommodate up to 500 participants in a theater-style setup, 750 in a reception-style setup, 320 in a banquet-style setup, 220 in a classroom-style setup, and 160 in a crescent-style setup.

The website mentions that the venue has one of the City's finest commercial kitchens and offers gourmet food and beverage services. Additionally, audio-visual services are available for events. To coordinate catering and equipment setup seamlessly for the event on 2024-09-15 at the Julia Morgan Ballroom, it is recommended to contact the venue directly for more information and to discuss specific requirements and options. 

Contact Information:
Julia Morgan Ballroom
The Merchants Exchange Building
465 Ca

Exception in thread Thread-5 (_execute):
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/usr/local/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/usr/local/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.11/site-packages/crewai/task.py", line 182, in _execute
    result = agent.execute_task(
             ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/site-packages/crewai/agent.py", line 221, in execute_task
    result = self.agent_executor.invoke(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/site-packages/langchain/chains/base.py", line 163, in invoke
    raise e
  File "/usr/local/lib/python3.11/site-packages/langchain/chains/base.py", line 153, in invoke
    self._call(inputs, run_manager=run_manager)
  File "/us

I need to gather information on potential marketing activities and attendee engagement for the Tech Innovation Conference.

Action: Search the internet
Action Input: {
    "search_query": "Tech Innovation Conference marketing activities"
} 


Search results: Title: Host a Winning Tech Conference: Complete Checklist - Swoogo
Link: https://swoogo.events/blog/hosting-a-conference-technology-industry/
Snippet: Make your conference pop with hands-on workshops and interactive sessions. Think coding bootcamps, panel discussions, and live Q&As.
---
Title: 15 conference marketing ideas for 2024 that really work | Kaltura
Link: https://corp.kaltura.com/blog/conference-marketing/
Snippet: Explore conference marketing benefits and effective strategies, including virtual event tips, to boost engagement and success in 2024.
---
Title: 13 Innovative Event Marketing Ideas
Link: https://blog.walls.io/events/event-marketing-ideas/
Snippet: Consider incorporating engaging onsite experiences like social m

Final Answer: 

# Marketing Activities for Tech Innovation Conference

## Overview
The Tech Innovation Conference aims to engage at least 500 potential attendees. To achieve this goal, a comprehensive marketing strategy and attendee engagement plan have been outlined based on research and best practices.

## Marketing Activities
1. **Social Media Promotion**: Utilize social media platforms to create buzz and drive engagement. Regular updates, behind-the-scenes content, and speaker announcements will be shared using a dedicated conference hashtag.
2. **Email Marketing**: Leverage email campaigns to reach a targeted audience. Highlight early bird discounts, key speakers, and detailed agendas to encourage registrations.
3. **Partner with Industry Influencers**: Collaborate with industry influencers to expand reach and credibility. Offer incentives such as free tickets or exclusive access.
4. **Event Management Software**: Utilize event management software for seamless organization. Automa

- Display the generated `venue_details.json` file.

In [15]:
import json
from pprint import pprint

with open('venue_details.json') as f:
   data = json.load(f)

pprint(data)

{'address': 'San Francisco, CA',
 'booking_status': 'Available',
 'capacity': 500,
 'name': 'Julia Morgan Ballroom'}


- Display the generated `marketing_report.md` file.

**Note**: After `kickoff` execution has successfully ran, wait an extra 45 seconds for the `marketing_report.md` file to be generated. If you try to run the code below before the file has been generated, your output would look like:

```
marketing_report.md
```

If you see this output, wait some more and than try again.

In [16]:
from IPython.display import Markdown
Markdown(marketing_report.md")

marketing_report.md

In [17]:
with open('venue_details.json') as f:
    line = f.readlines()
    print(data)

{'name': 'Julia Morgan Ballroom', 'address': 'San Francisco, CA', 'capacity': 500, 'booking_status': 'Available'}
